## author: Ryan Inghilterra

want to be able to easily join all datasets on id, so have all info per sample_id

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 200)
pd.options.display.max_seq_items = 200

**DIRECTORY WHERE TO EXPORT ALL CLEANED UP DATA**

In [3]:
output_dir = 'join_wflow_output_data/'

### read in metadata

In [4]:
meta_df = pd.read_csv('raw.2.21.agp_metadata.txt',sep='\t', low_memory=False)

In [5]:
meta_df.shape

(26009, 597)

In [6]:
meta_df.head(3)

,sample_name,acid_reflux,acne_medication,acne_medication_otc,add_adhd,age_cat,age_corrected,age_years,alcohol_consumption,alcohol_frequency,alcohol_types,alcohol_types_beercider,alcohol_types_red_wine,alcohol_types_sour_beers,alcohol_types_spiritshard_alcohol,alcohol_types_unspecified,alcohol_types_white_wine,allergic_to,allergic_to_i_have_no_food_allergies_that_i_know_of,allergic_to_other,allergic_to_peanuts,allergic_to_shellfish,allergic_to_tree_nuts,allergic_to_unspecified,altitude,alzheimers,animal_age,animal_free_text,animal_gender,animal_origin,animal_type,anonymized_name,antibiotic_history,appendix_removed,artificial_sweeteners,asd,assigned_from_geo,autoimmune,birth_year,bmi,bmi_cat,bmi_corrected,body_habitat,body_product,body_site,bowel_movement_frequency,bowel_movement_quality,breastmilk_formula_ensure,cancer,cancer_treatment,...,vioscreen_tgrain,vioscreen_thiamin,vioscreen_threonin,vioscreen_time,vioscreen_totaltfa,vioscreen_totcla,vioscreen_totfolat,vioscreen_totsugar,vioscreen_tryptoph,vioscreen_tyrosine,vioscreen_user_id,vioscreen_v_drkgr,vioscreen_v_orange,vioscreen_v_other,vioscreen_v_potato,vioscreen_v_starcy,vioscreen_v_tomato,vioscreen_v_total,vioscreen_valine,vioscreen_veg5_day,vioscreen_vegetable_servings,vioscreen_vegsumm,vioscreen_visit,vioscreen_vita_iu,vioscreen_vita_rae,vioscreen_vita_re,vioscreen_vitb12,vioscreen_vitb6,vioscreen_vitc,vioscreen_vitd,vioscreen_vitd2,vioscreen_vitd3,vioscreen_vitd_iu,vioscreen_vite_iu,vioscreen_vitk,vioscreen_water,vioscreen_weight,vioscreen_wgrain,vioscreen_whole_grain_servings,vioscreen_xylitol,vioscreen_zinc,vitamin_b_supplement_frequency,vitamin_d_supplement_frequency,vivid_dreams,weight_cat,weight_change,weight_kg,weight_units,whole_eggs,whole_grain_frequency
0,10317.000001000,Not provided,false,false,"Diagnosed by a medical professional (doctor, p...",60s,64.0,64,true,Daily,NaN,true,false,false,true,false,false,NaN,false,true,false,false,false,false,Not applicable,I do not have this condition,NaN,NaN,NaN,NaN,NaN,000001000,6 months,true,Not provided,I do not have this condition,Not applicable,I do not have this condition,1952,21.92,Normal,21.92,UBERON:feces,UBERON:feces,UBERON:feces,Two,Not provided,Not provided,Not provided,Not provided,...,Not provided,Not provided,Not provided,NaN,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,NaN,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,NaN,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,NaN,Not provided,Not provided,Not provided,Not provided,Never,Regularly (3-5 times/week),Not provided,NaN,Remained stable,52,kilograms,Never,Occasionally (1-2 times/week)
1,10317.000001001,Not provided,false,false,Not provided,50s,53.0,53.0,true,Rarely (a few times/month),NaN,false,false,false,false,true,false,NaN,false,false,false,false,false,true,Not applicable,Not provided,NaN,NaN,NaN,NaN,NaN,000001001,I have not taken antibiotics in the past year.,false,Not provided,Not provided,Not applicable,Not provided,1962.0,35.92,Obese,Not provided,UBERON:feces,UBERON:feces,UBERON:feces,Not provided,Not provided,Not provided,Not provided,Not provided,...,Not provided,Not provided,Not provided,NaN,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,NaN,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,NaN,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,NaN,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,NaN,Remained stable,110.0,kilograms,Not provided,Not provided
2,10317.000001002,Not provided,false,false,Not provided,50s,53.

### filter on only body_site = FECES

In [7]:
meta_df = meta_df[meta_df.body_site == 'UBERON:feces']
print(meta_df.shape)

(19604, 597)


In [8]:
#meta_df.columns

In [9]:
meta_df[['sample_name', 'survey_id', 'qiita_study_id']].head(2)

,sample_name,survey_id,qiita_study_id
0,10317.000001000,48c83b76031b7580,10317
1,10317.000001001,0b0832aafb76068d,10317


In [10]:
meta_df[['sample_name', 'survey_id']].sort_values('sample_name').head(2)

,sample_name,survey_id
0,10317.000001000,48c83b76031b7580
1,10317.000001001,0b0832aafb76068d


In [11]:
meta_df[meta_df['sample_name'].str.contains('Blank')].shape

(0, 597)

### read in drug data dense

In [12]:
drug_df = pd.read_csv('../drug_data/drugbank_drug_data.csv')
print(drug_df.shape)

(53271, 11)


In [13]:
drug_df.head(2)

,Unnamed: 0,sample_name,survey_id,question_shortname,response,match_col_count,drug_matches,ATC1,Description,Name,Fecal
0,0,72291,0000904d9779a86c,MEDICATION_LIST,"Pravastatin, CoQ10",2,C10AA03,C,Cardiovascular system,Pravastatin,False
1,1,12914,000a1da7d9d7e35b,ANTIBIOTIC_MED,Zymar (gatifloxacin) eye drop (NOT ORAL),1,J01MA16,J,Antiinfectives for systemic use,Gatifloxacin,NaN


In [14]:
drug_df.sort_values('sample_name').head(2)

,Unnamed: 0,sample_name,survey_id,question_shortname,response,match_col_count,drug_matches,ATC1,Description,Name,Fecal
12666,12666,1000,48c83b76031b7580,MEDICATION_LIST,lexapro amitriptoline welbutrin,3,N06AB10,N,Nervous system,Escitalopram,False
12668,12668,1000,48c83b76031b7580,MEDICATION_LIST,lexapro amitriptoline welbutrin,3,N06AX12,N,Nervous system,Bupropion,False


### create dataframes with joinable ids

In [15]:
def meta_df_id_clean(meta_df):
    meta_df['sample_id'] = meta_df['sample_name'].apply(lambda x: x.split('.')[1]) 
    meta_df['sample_id'] = pd.to_numeric(meta_df['sample_id'], errors='coerce', downcast='integer')
    meta_df_clean = meta_df.dropna(subset=['sample_id']).reset_index()
    meta_df_clean['sample_id'] = meta_df_clean['sample_id'].apply(lambda x: int(x))
    return meta_df_clean

In [16]:
def drug_df_id_clean(drug_df):
    drug_df['sample_id'] = drug_df['sample_name'].apply(lambda x: str(x).split('.')[0])
    drug_df['sample_id'] = drug_df['sample_id'].astype('int')
    return drug_df

clean metadata sample_id that will match with drug_id

In [17]:
meta_clean_id_df = meta_df_id_clean(meta_df)

In [18]:
print(meta_clean_id_df.shape)
meta_clean_id_df[['sample_id', 'sample_name']].head(2)

(19571, 599)


,sample_id,sample_name
0,1000,10317.000001000
1,1001,10317.000001001


clean drug_df

In [19]:
drug_clean_id_df = drug_df_id_clean(drug_df)

In [20]:
print(drug_clean_id_df.shape)
drug_clean_id_df.head(2)

(53271, 12)


,Unnamed: 0,sample_name,survey_id,question_shortname,response,match_col_count,drug_matches,ATC1,Description,Name,Fecal,sample_id
0,0,72291,0000904d9779a86c,MEDICATION_LIST,"Pravastatin, CoQ10",2,C10AA03,C,Cardiovascular system,Pravastatin,False,72291
1,1,12914,000a1da7d9d7e35b,ANTIBIOTIC_MED,Zymar (gatifloxacin) eye drop (NOT ORAL),1,J01MA16,J,Antiinfectives for systemic use,Gatifloxacin,NaN,12914


try joing drug_data and metadata on id

In [21]:
merge_df = pd.merge(meta_clean_id_df[['sample_name', 'sample_id']], drug_clean_id_df, how='left', on='sample_id')

In [22]:
merge_df.shape

(50519, 13)

In [23]:
merge_df.head()

,sample_name_x,sample_id,Unnamed: 0,sample_name_y,survey_id,question_shortname,response,match_col_count,drug_matches,ATC1,Description,Name,Fecal
0,10317.000001000,1000,12666.0,1000.0,48c83b76031b7580,MEDICATION_LIST,lexapro amitriptoline welbutrin,3.0,N06AB10,N,Nervous system,Escitalopram,False
1,10317.000001000,1000,12667.0,1000.0,48c83b76031b7580,MEDICATION_LIST,lexapro amitriptoline welbutrin,3.0,N06AA09,N,Nervous system,Amitriptyline,True
2,10317.000001000,1000,12668.0,1000.0,48c83b76031b7580,MEDICATION_LIST,lexapro amitriptoline welbutrin,3.0,N06AX12,N,Nervous system,Bupropion,False
3,10317.000001001,1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10317.000001002,1002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


lets split vioscreen and metadata columns

In [24]:
col_names = pd.Series(meta_clean_id_df.columns)
meta_col_names = list(col_names[~col_names.str.contains('vioscreen')])
vio_col_names = list(col_names[col_names.str.contains('vioscreen')])
vio_col_names = vio_col_names + ['sample_name', 'sample_id', 'survey_id']

In [25]:
print(len(meta_col_names))
print(len(vio_col_names))

326
276


In [26]:
meta_agp_df = meta_clean_id_df[meta_col_names]
print(meta_agp_df.shape)
meta_agp_df[['sample_id']].head(2)

(19571, 326)


,sample_id
0,1000
1,1001


In [27]:
vio_df = meta_clean_id_df[vio_col_names]
print(vio_df.shape)
vio_df.head(2)

(19571, 276)


,vioscreen_a_bev,vioscreen_a_cal,vioscreen_acesupot,vioscreen_activity_level,vioscreen_add_sug,vioscreen_addsugar,vioscreen_adsugtot,vioscreen_age,vioscreen_alanine,vioscreen_alcohol,vioscreen_alcohol_servings,vioscreen_alphacar,vioscreen_alphtoce,vioscreen_alphtoco,vioscreen_arginine,vioscreen_ash,vioscreen_aspartam,vioscreen_aspartic,vioscreen_avcarb,vioscreen_bcodeid,vioscreen_betacar,vioscreen_betacryp,vioscreen_betaine,vioscreen_betatoco,vioscreen_biochana,vioscreen_bmi,vioscreen_caffeine,vioscreen_calcium,vioscreen_calcium_avg,vioscreen_calcium_dose,vioscreen_calcium_freq,vioscreen_calcium_from_dairy_servings,vioscreen_calcium_servings,vioscreen_calories,vioscreen_carbo,vioscreen_cholest,vioscreen_choline,vioscreen_clac9t11,vioscreen_clat10c12,vioscreen_copper,vioscreen_coumest,vioscreen_cystine,vioscreen_d_cheese,vioscreen_d_milk,vioscreen_d_tot_soym,vioscreen_d_total,vioscreen_d_yogurt,vioscreen_daidzein,vioscreen_database,vioscreen_delttoco,...,vioscreen_sucrose,vioscreen_sweet_servings,vioscreen_tagatose,vioscreen_tfa161t,vioscreen_tfa181t,vioscreen_tfa182t,vioscreen_tgrain,vioscreen_thiamin,vioscreen_threonin,vioscreen_time,vioscreen_totaltfa,vioscreen_totcla,vioscreen_totfolat,vioscreen_totsugar,vioscreen_tryptoph,vioscreen_tyrosine,vioscreen_user_id,vioscreen_v_drkgr,vioscreen_v_orange,vioscreen_v_other,vioscreen_v_potato,vioscreen_v_starcy,vioscreen_v_tomato,vioscreen_v_total,vioscreen_valine,vioscreen_veg5_day,vioscreen_vegetable_servings,vioscreen_vegsumm,vioscreen_visit,vioscreen_vita_iu,vioscreen_vita_rae,vioscreen_vita_re,vioscreen_vitb12,vioscreen_vitb6,vioscreen_vitc,vioscreen_vitd,vioscreen_vitd2,vioscreen_vitd3,vioscreen_vitd_iu,vioscreen_vite_iu,vioscreen_vitk,vioscreen_water,vioscreen_weight,vioscreen_wgrain,vioscreen_whole_grain_servings,vioscreen_xylitol,vioscreen_zinc,sample_name,sample_id,survey_id
0,Not provided,Not provided,Not provided,NaN,Not provided,Not provided,Not provided,NaN,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,NaN,Not provided,Not provided,Not provided,Not provided,Not provided,NaN,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,...,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,NaN,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,NaN,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,NaN,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,NaN,Not provided,Not provided,Not provided,Not provided,10317.000001000,1000,48c83b76031b7580
1,Not provided,Not provided,Not provided,NaN,Not provided,Not provided,Not provided,NaN,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,NaN,Not provided,Not provided,Not provided,Not provided,Not provided,NaN,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,...,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,NaN,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,NaN,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not pro

test merges to make sure works before exporting datasets to csv

In [28]:
merge_df2 = pd.merge(meta_agp_df[['sample_name', 'sample_id']], drug_clean_id_df, how='left', on='sample_id')
merge_df2.shape
merge_df2.head(2)

,sample_name_x,sample_id,Unnamed: 0,sample_name_y,survey_id,question_shortname,response,match_col_count,drug_matches,ATC1,Description,Name,Fecal
0,10317.000001000,1000,12666.0,1000.0,48c83b76031b7580,MEDICATION_LIST,lexapro amitriptoline welbutrin,3.0,N06AB10,N,Nervous system,Escitalopram,False
1,10317.000001000,1000,12667.0,1000.0,48c83b76031b7580,MEDICATION_LIST,lexapro amitriptoline welbutrin,3.0,N06AA09,N,Nervous system,Amitriptyline,True


In [29]:
merge_df3 = pd.merge(vio_df[['sample_name', 'sample_id']], drug_clean_id_df, how='left', on='sample_id')
merge_df3.shape
merge_df3.head(2)

,sample_name_x,sample_id,Unnamed: 0,sample_name_y,survey_id,question_shortname,response,match_col_count,drug_matches,ATC1,Description,Name,Fecal
0,10317.000001000,1000,12666.0,1000.0,48c83b76031b7580,MEDICATION_LIST,lexapro amitriptoline welbutrin,3.0,N06AB10,N,Nervous system,Escitalopram,False
1,10317.000001000,1000,12667.0,1000.0,48c83b76031b7580,MEDICATION_LIST,lexapro amitriptoline welbutrin,3.0,N06AA09,N,Nervous system,Amitriptyline,True


export to .csv

In [228]:
meta_agp_df.to_csv(output_dir + '2.21.agp_only_meta.csv', index=False)

In [229]:
vio_df.to_csv(output_dir + '2.21.vio_only.csv', index=False)

export versions of drug data to .csv

In [230]:
drug_df = pd.read_csv('../drug_data/drugbank_drug_data.csv')
drug_clean_id_df = drug_df_id_clean(drug_df)
print(drug_clean_id_df.shape)

(53271, 12)


In [231]:
drug_clean_id_df[['sample_id']].head(2)

,sample_id
0,72291
1,12914


In [232]:
drug_clean_id_df.to_csv(output_dir + '2.21.drugbank_drug_data.csv', index=False)

In [233]:
drug_df = pd.read_csv('../drug_data/drug_data_dense.csv')
drug_clean_id_df = drug_df_id_clean(drug_df)
print(drug_clean_id_df.shape)

(23491, 7)


In [234]:
drug_clean_id_df[['sample_id']].head(2)

,sample_id
0,72291
1,43062


In [235]:
drug_clean_id_df.to_csv(output_dir + '2.21.drug_data_dense.csv', index=False)

## biom table data join

In [30]:
biom_df = pd.read_pickle('deblur_biom_wflow_output_data/2.21.rar1000.biom_data.pkl')

In [31]:
print(type(biom_df))
biom_df.shape

<class 'pandas.core.sparse.frame.SparseDataFrame'>


(18305, 111508)

In [32]:
#biom_df.head(3)

In [33]:
biom_df['sample_id'] = biom_df['sample_name'].apply(lambda x: x.split('.')[1]) 

biom_df['sample_id'] = pd.to_numeric(biom_df['sample_id'], errors='coerce', downcast='integer')

biom_join_df = biom_df.dropna(subset=['sample_id'])

biom_join_df['sample_id'] = biom_join_df['sample_id'].apply(lambda x: int(x))

//anaconda/envs/qiime2-2018.11/lib/python3.5/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [34]:
print(biom_join_df.shape)

(18273, 111509)


In [35]:
#biom_join_df.head()

#### test merge between biom data and metadata where we do left join from metadata

In [36]:
merge_df4 = pd.merge(meta_agp_df[['sample_name', 'sample_id']], 
                     biom_join_df[['sample_name', 'sample_id']], how='left', on='sample_id')
print(merge_df4.shape)
merge_df4.head(2)

(20731, 3)


,sample_name_x,sample_id,sample_name_y
0,10317.000001000,1000,10317.000001000.60809
1,10317.000001001,1001,10317.000001001.59277


In [37]:
merge_df4.isna().sum()

sample_name_x       0
sample_id           0
sample_name_y    4613
dtype: int64

any nulls above in sample_name_y are the number of missing sample_ids in biom OTU df from samples in the metadata, it is expected there are some since the biom df is not filtered on feces only

In [39]:
merge4_df = merge_df4.dropna()
print(merge4_df.shape)

(16118, 3)


we want to filter the biom dataframe on sample_ids only found in our cleaned up metadata df

In [44]:
final_biom_df = biom_join_df[biom_join_df['sample_id'].isin(merge4_df['sample_id'])]

In [45]:
print(final_biom_df.shape)

(16118, 111509)


In [244]:
#biom_join_df.head()

#### looking good!

change name of pkl file depending on date and rar sample level

In [46]:
final_biom_df.to_pickle(output_dir + '2.21.rar1000_clean_biom.pkl')

In [47]:
bdf = pd.read_pickle(output_dir + '2.21.rar1000_clean_biom.pkl')

In [48]:
print(bdf.shape)

(16118, 111509)


In [248]:
#bdf.head(1)

In [249]:
print(bdf.shape)

(18273, 111509)


**small sample to test alpha div**

In [250]:
sample1 = bdf.head(5)

In [251]:
sample1 = sample1.set_index('sample_id')

In [252]:
del sample1['sample_name']

In [253]:
112015 - sample1.isna().sum(axis=1)

sample_id
67572    604
35170    585
97270    616
41721    705
93557    643
dtype: int64

number of unique OTUs in these 5 samples. (Non null cols)